In [353]:
function MIN-CONFLICTS(csp, max_steps) returns a solution of failure 
    inputs: csp, a constraint satisfaction problem    
        max_steps, the number of steps allowed before giving up 
    current ← an initial complete assignment for csp 
    for i = 1 to max_steps do   
        if current is a solution for csp then return current   
        var ← a randomly chosen conflicted variable from csp.VARIABLES   
        value ← the value v for var that minimizes CONFLICTS(var, v, current, csp)   
        set var = value in current 
return failure

SyntaxError: invalid character '←' (U+2190) (3344788426.py, line 4)

In [595]:
import random
import copy

class SudokuCSP:
    def __init__(self, puzzle):
        self.board = [[int(puzzle[i*9 + j]) for j in range(9)] for i in range(9)]
        self.initialize_board()

    def display(self):
        for row in self.board:
            print(" ".join(str(val) if val != 0 else '.' for val in row))

    def is_consistent(self, row, col, val):
        for i in range(9):
            if self.board[row][i] == val or self.board[i][col] == val:
                return False
        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(start_row, start_row + 3):
            for j in range(start_col, start_col + 3):
                if self.board[i][j] == val:
                    return False
        return True

    def initialize_board(self):
        for i in range(9):
            for j in range(9):
                if self.board[i][j] == 0:
                    possible_values = [val for val in range(1, 10) if self.is_consistent(i, j, val)]
                    self.board[i][j] = random.choice(possible_values) if possible_values else 0

    def count_conflicts(self, row, col):
        conflicts = 0
        val = self.board[row][col]
        for i in range(9):
            if i != col and self.board[row][i] == val:
                conflicts += 1
            if i != row and self.board[i][col] == val:
                conflicts += 1
        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(start_row, start_row + 3):
            for j in range(start_col, start_col + 3):
                if (i != row or j != col) and self.board[i][j] == val:
                    conflicts += 1
        return conflicts

    def find_most_conflicting_cell(self):
        max_conflicts = 0
        cell = None
        for i in range(9):
            for j in range(9):
                if self.board[i][j] != 0:
                    conflicts = self.count_conflicts(i, j)
                    if conflicts > max_conflicts:
                        max_conflicts = conflicts
                        cell = (i, j)
        return cell

    def solve(self):
        best_board = None
        min_conflicts = float('inf')
        iterations = 0
        while iterations < 100000:  # Adjusted iteration limit
            cell = self.find_most_conflicting_cell()
            if not cell and self.is_complete():
                return True  # Solved and complete

            if iterations % 100 == 0:  # Adjusted for more frequent shuffling
                self.shuffle_board()

            total_conflicts = sum(self.count_conflicts(i, j) for i in range(9) for j in range(9) if self.board[i][j] != 0)
            if total_conflicts < min_conflicts:
                min_conflicts = total_conflicts
                best_board = copy.deepcopy(self.board)

            if iterations % 500 == 0 and total_conflicts > 0:  # Adjusted restart strategy
                self.initialize_board()

            iterations += 1

        if min_conflicts == 0 and self.is_complete(best_board):
            self.board = best_board
            return True

        return False  # No solution found within the iteration limit

    def is_complete(self, board=None):
        if board is None:
            board = self.board
        for row in board:
            if 0 in row:
                return False
        return True

    def shuffle_board(self):
        choice = random.choice(["row", "column", "box"])
        if choice == "row":
            self.shuffle_row(random.randint(0, 8))
        elif choice == "column":
            self.shuffle_column(random.randint(0, 8))
        else:
            self.shuffle_box(random.randint(0, 2), random.randint(0, 2))

    def shuffle_row(self, row):
        initial_state = self.board[row]
        random.shuffle(self.board[row])
        for col in range(9):
            if not self.is_consistent(row, col, self.board[row][col]) or self.board[row][col] == initial_state[col]:
                self.board[row] = initial_state
                break

    def shuffle_column(self, col):
        initial_state = [self.board[row][col] for row in range(9)]
        shuffled = initial_state[:]
        random.shuffle(shuffled)
        for row in range(9):
            self.board[row][col] = shuffled[row]
            if not self.is_consistent(row, col, self.board[row][col]) or self.board[row][col] == initial_state[row]:
                for r in range(9):
                    self.board[r][col] = initial_state[r]
                break

    def shuffle_box(self, box_row, box_col):
        initial_state = []
        for row in range(3):
            for col in range(3):
                initial_state.append(self.board[box_row * 3 + row][box_col * 3 + col])
        shuffled = initial_state[:]
        random.shuffle(shuffled)
        idx = 0
        for row in range(3):
            for col in range(3):
                self.board[box_row * 3 + row][box_col * 3 + col] = shuffled[idx]
                idx += 1
                if not self.is_consistent(box_row * 3 + row, box_col * 3 + col, self.board[box_row * 3 + row][box_col * 3 + col]):
                    idx = 0
                    for r in range(3):
                        for c in range(3):
                            self.board[box_row * 3 + r][box_col * 3 + c] = initial_state[idx]
                            idx += 1
                    return

# Example usage
#sudoku_problem = '000395172137024950259167834318609725925783461476251389891532647543976218762418500'
sudoku_problem = '000005000037800006200167800300009720905783060470200089890502600540900000000400593'
# sudoku_problem_hard = '800600053000000040063800000032000005000007000406002089000060704000018000609700018'
sudoku = SudokuCSP(sudoku_problem)
sudoku.display()

if sudoku.solve():
    print("A solution was found:\n")
    sudoku.display()
else:
    print("Failed to find a solution.")


6 1 9 3 2 5 4 7 .
. 3 7 8 9 4 1 5 6
2 5 4 1 6 7 8 3 .
3 6 1 . 4 9 7 2 5
9 2 5 7 8 3 . 6 1
4 7 . 2 1 6 3 8 9
8 9 3 5 7 2 6 4 .
5 4 6 9 3 1 2 . 8
1 . 2 4 . 8 5 9 3
Failed to find a solution.
